In [1]:
import os
import datetime
import re
import requests
import pandas as pd

from dotenv import load_dotenv
from time import perf_counter
from tqdm import tqdm
from requests_ntlm import HttpNtlmAuth
from itertools import groupby
from io import BytesIO

from fetchbim.teams import sync_complete_msg
from notion_client import Client as NotionClient

In [2]:
load_dotenv()

current_year = datetime.datetime.now().year
trim = re.compile(r'[^\d.]+')
user = os.getenv('REPORT_SERVER_USER')
auth = HttpNtlmAuth(os.getenv('REPORT_SERVER_USER'), os.getenv('REPORT_SERVER_PASS'))

In [3]:
r = requests.get(
    r'http://reporting.southwestsolutions.com/ReportServer?%2FBIM%2FVendor%20Metrics&rs:Format=CSV', 
    auth=auth
)

In [4]:
df = pd.read_csv(BytesIO(r.content))
df.head()

,Year,VendorID,Vendor,City,State,Zipcode,Costs,OrderCount
0,2023,35867,Datum Filing Systems Inc.,Emigsville,PA,17318,"$6,407.69",3
1,2023,32736,Hallowell,Atlanta,GA,30374,"$2,613.30",1
2,2023,28552,Hamilton Casework Solutions,Fairfield,OH,45014,"$1,131.00",1
3,2023,36658,Equipto,Tatamy,PA,18085,$957.07,1
4,2023,30957,Star Filing,Atlanta,GA,30336,$645.00,1


In [5]:
gk = df.groupby("VendorID")
gk.first()

,Year,Vendor,City,State,Zipcode,Costs,OrderCount
VendorID,,,,,,,
22207,2021,Furniture Marketing Group (Facility Interiors),Austin,TX,78705,$300.00,1
22349,2022,"Pegasus Logistics Group, Inc.",Coppell,TX,75019,"$11,068.63",3
27401,2022,Wood Design,Fort Atkinson,WI,53538,"$43,442.27",5
27416,2022,Associated Office Systems,New Orleans,LA,70130,"$24,425.00",1
27440,2022,"Aurora Storage Products, Inc.",Aurora,IL,60506,"$98,298.17",15
...,...,...,...,...,...,...,...
347843,2022,Match Freight Lines,Beaverton,OR,97008,$903.71,1
348198,2021,Countertop Company and Tile Installations,Forney,TX,75126,"$1,290.00",1
348398,2022,Anchor Manufacturing,Fulshear,TX,77441,"$9,541.02",1


In [6]:
gk.get_group(28552)

,Year,VendorID,Vendor,City,State,Zipcode,Costs,OrderCount
2,2023,28552,Hamilton Casework Solutions,Fairfield,OH,45014,"$1,131.00",1
14,2022,28552,Hamilton Casework Solutions,Fairfield,OH,45014,"$1,144,839.86",155
309,2021,28552,Hamilton Casework Solutions,Fairfield,OH,45014,"$760,501.42",154
642,2020,28552,Hamilton Casework Solutions,Fairfield,OH,45014,"$591,546.95",141


In [18]:
df.loc[:,['Year', 'Vendor', 'Costs']]

,Year,Vendor,Costs
0,2023,Datum Filing Systems Inc.,"$6,407.69"
1,2023,Hallowell,"$2,613.30"
2,2023,Hamilton Casework Solutions,"$1,131.00"
3,2023,Equipto,$957.07
4,2023,Star Filing,$645.00
...,...,...,...
964,2020,Kuster Sign Co.,$50.00
965,2020,Wanzl North America,$45.48
966,2020,Mountain Moving & Storage,$7.90
967,2020,Palmieri,$0.00


In [19]:
for g, k in gk:
    if g == 28552:
        print(k.loc[:,['Year', 'Vendor', 'Costs']])
        break
    # print(f"Vendor={k['Vendor']}, Year={k['Year']}, Costs={k['Costs']}, OrderCount={k['OrderCount']}")

     Year                       Vendor          Costs
2    2023  Hamilton Casework Solutions      $1,131.00
14   2022  Hamilton Casework Solutions  $1,144,839.86
309  2021  Hamilton Casework Solutions    $760,501.42
642  2020  Hamilton Casework Solutions    $591,546.95


In [20]:
df.loc[df.VendorID == 28552]

,Year,VendorID,Vendor,City,State,Zipcode,Costs,OrderCount
2,2023,28552,Hamilton Casework Solutions,Fairfield,OH,45014,"$1,131.00",1
14,2022,28552,Hamilton Casework Solutions,Fairfield,OH,45014,"$1,144,839.86",155
309,2021,28552,Hamilton Casework Solutions,Fairfield,OH,45014,"$760,501.42",154
642,2020,28552,Hamilton Casework Solutions,Fairfield,OH,45014,"$591,546.95",141
